### You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/gobot_extended_tutorial.ipynb).

In [ ]:
!pip install deeppavlov

# Goal-oriented bot in DeepPavlov

The tutor is focused on building a goal-oriented dialogue system:

0. [Data preparation](#0.-Data-Preparation)
1. [Build database of items](#1.-Build-database-of-items)
2. [Build Slot Filler](#2.-Build-Slot-Filler)
3. [Train bot](#3.-Train-bot)

An example of the final model served as a telegram bot is:

![gobot_example.png](img/gobot_example.png)

## 0. Data Preparation

The tutor's dialogue system will be on the domain of restaurant booking. [Dialogue State Tracking Challenge 2 (DSTC-2)](http://camdial.org/~mh521/dstc/) dataset provides dialogues of a human talking to a booking system labelled with slots and dialogue actions. The labels are will be used for training a dialogue policy network.

See below a small chunk of the data. 

In [2]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader

data = SimpleDSTC2DatasetReader().read('my_data')

2019-09-05 20:02:11.725 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 269: [PosixPath('my_data/simple-dstc2-val.json'), PosixPath('my_data/simple-dstc2-tst.json')]]
2019-09-05 20:02:11.725 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 270: [downloading data from http://files.deeppavlov.ai/datasets/simple_dstc2.tar.gz to my_data]
2019-09-05 20:02:11.775 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/simple_dstc2.tar.gz to my_data/simple_dstc2.tar.gz
100%|██████████| 497k/497k [00:00<00:00, 44.6MB/s]
2019-09-05 20:02:11.816 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_data/simple_dstc2.tar.gz archive into my_data
2019-09-05 20:02:11.841 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from my_data/simple-dstc2-trn.json]
2019-09-05 20:02:11.892 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['ds

In [3]:
!ls my_data

simple-dstc2-templates.txt  simple-dstc2-tst.json
simple-dstc2-trn.json	    simple-dstc2-val.json


The training/validation/test data is stored in json files (`simple-dstc2-trn.json`, `simple-dstc2-val.json` and `simple-dstc2-tst.json`):

In [4]:
!head -n 101 my_data/simple-dstc2-trn.json

[
  [
    {
      "speaker": 2,
      "text": "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?",
      "slots": [],
      "act": "welcomemsg"
    },
    {
      "speaker": 1,
      "text": "cheap restaurant",
      "slots": [
        [
          "pricerange",
          "cheap"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What kind of food would you like?",
      "slots": [],
      "act": "request_food"
    },
    {
      "speaker": 1,
      "text": "any",
      "slots": [
        [
          "this",
          "dontcare"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What part of town do you have in mind?",
      "slots": [],
      "act": "request_area"
    },
    {
      "speaker": 1,
      "text": "south",
      "slots": [
        [
          "area",
          "south"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "api_call area=\"south\" food

In [5]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

You can now iterate over batches of preprocessed DSTC-2 dialogs:

In [6]:
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1, data_type='train'):
    turns_x, turns_y = dialog
    
    print("User utterances:\n----------------\n")
    pprint(turns_x[0], indent=4)
    print("\nSystem responses:\n-----------------\n")
    pprint(turns_y[0], indent=4)
    
    break

User utterances:
----------------

[   {'prev_resp_act': None, 'text': ''},
    {'prev_resp_act': 'welcomemsg', 'text': 'sil'},
    {   'prev_resp_act': 'request_area',
        'slots': [['area', 'east']],
        'text': 'east'},
    {   'db_result': {   'addr': '71 castle street city centre',
                         'area': 'west',
                         'food': 'indian',
                         'name': 'cocum',
                         'phone': '01223 366668',
                         'postcode': 'c.b 3, 0 a.h',
                         'pricerange': 'expensive'},
        'prev_resp_act': 'api_call',
        'slots': [['area', 'east']],
        'text': 'east'},
    {   'prev_resp_act': 'offer_name',
        'slots': [['pricerange', 'expensive']],
        'text': 'expensive restaurant'},
    {   'prev_resp_act': 'inform_pricerange+offer_name',
        'slots': [['slot', 'addr']],
        'text': 'whats the address'},
    {   'prev_resp_act': 'inform_addr+offer_name',
        'slo

In [7]:
!cp my_data/simple-dstc2-trn.json my_data/simple-dstc2-trn.full.json

In [8]:
import json

NUM_TRAIN = 50

with open('my_data/simple-dstc2-trn.full.json', 'rt') as fin:
    data = json.load(fin)
with open('my_data/simple-dstc2-trn.json', 'wt') as fout:
    json.dump(data[:NUM_TRAIN], fout, indent=2)
print(f"Train set is reduced to {NUM_TRAIN} dialogues (out of {len(data)}).")

Train set is reduced to 50 dialogues (out of 967).


## 1. Build database of items

&nbsp;
![gobot_database.png](img/gobot_database.png)
&nbsp;

For a valid goal-oriented bot there should be a `database` of relevant items. In the case of restaurant booking it will contain all available restaurants and their info.

    >> database([{'pricerange': 'cheap', 'area': 'south'}])
    
    Out[1]: 
        [[{'name': 'the lucky star',
           'food': 'chinese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way cherry hinton',
           'phone': '01223 244277',
           'postcode': 'c.b 1, 7 d.y'},
          {'name': 'nandos',
           'food': 'portuguese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way',
           'phone': '01223 327908',
           'postcode': 'c.b 1, 7 d.y'}]]
           
The dialogues in the training dataset should contain a `"db_result"` dictionary key. It is required for turns where system performs a special type of external action: an api call to the database of items. `"db_result"` should contain the result of the api call:

In [9]:
!head -n 78 my_data/simple-dstc2-trn.json | tail +51

    {
      "speaker": 2,
      "text": "api_call area=\"south\" food=\"dontcare\" pricerange=\"cheap\"",
      "db_result": {
        "food": "chinese",
        "pricerange": "cheap",
        "area": "south",
        "addr": "cambridge leisure park clifton way cherry hinton",
        "phone": "01223 244277",
        "postcode": "c.b 1, 7 d.y",
        "name": "the lucky star"
      },
      "slots": [
        [
          "area",
          "south"
        ],
        [
          "pricerange",
          "cheap"
        ],
        [
          "food",
          "dontcare"
        ]
      ],
      "act": "api_call"
    },


In [10]:
from deeppavlov.core.data.sqlite_database import Sqlite3Database

database = Sqlite3Database(primary_keys=["name"],
                           save_path="my_bot/db.sqlite")

2019-09-05 20:02:20.149 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-05 20:02:20.150 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 70: Initializing empty database on /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.


Set `primary_keys` to a list of slot names that have unique values for different items (common SQL term). For the case of DSTC-2, the primary slot is restaurant name.

Let's find all `"db_result"` api call results and add it to our database of restaurants:

In [11]:
db_results = []

for dialog in iterator.gen_batches(batch_size=1, data_type='all'):
    turns_x, turns_y = dialog
    db_results.extend(x['db_result'] for x in turns_x[0] if x.get('db_result'))

print(f"Adding {len(db_results)} items.")
if db_results:
    database.fit(db_results)

2019-09-05 20:02:21.186 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 145: Created table with keys {'phone': 'text', 'name': 'text', 'food': 'text', 'area': 'text', 'addr': 'text', 'postcode': 'text', 'pricerange': 'text'}.


Adding 3016 items.


##### Interacting with database

We can now play with the database and make requests to it:

In [12]:
database([{'pricerange': 'cheap', 'area': 'south'}])

[[{'phone': '01223 327908',
   'name': 'nandos',
   'food': 'portuguese',
   'area': 'south',
   'addr': 'cambridge leisure park clifton way',
   'postcode': 'c.b 1, 7 d.y',
   'pricerange': 'cheap'},
  {'phone': '01223 244277',
   'name': 'the lucky star',
   'food': 'chinese',
   'area': 'south',
   'addr': 'cambridge leisure park clifton way cherry hinton',
   'postcode': 'c.b 1, 7 d.y',
   'pricerange': 'cheap'}]]

In [13]:
!ls my_bot

db.sqlite


## 2. Build Slot Filler

&nbsp;
![gobot_slotfiller.png](img/gobot_slotfiller.png)
&nbsp;

Slot Filler is component that inputs text and outputs dictionary of slot names and their values:

    >> slot_filler(['I would like some chineese food'])
    
    Out[1]: [{'food': 'chinese'}]

To implement a slot filler you need to provide
    
 - **slot types**
 - all possible **slot values**
 - optionally, it will be good to provide examples of mentions for every value of each slot
 
The data should be in `slot_vals.json` file with the following format:

    {
        'food': {
            'chinese': ['chinese', 'chineese', 'chines'],
            'french': ['french', 'freench'],
            'dontcare': ['any food', 'any type of food']
        }
    }
                

Let's use a simple non-trainable slot filler that relies on levenshtein distance:

In [14]:
from deeppavlov.download import download_decompress

download_decompress(url='http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz',
                    download_path='my_bot/slotfill')

2019-09-05 20:02:24.861 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz to my_bot/slotfill/dstc_slot_vals.tar.gz
100%|██████████| 1.62k/1.62k [00:00<00:00, 9.89MB/s]
2019-09-05 20:02:24.862 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_bot/slotfill/dstc_slot_vals.tar.gz archive into my_bot/slotfill


In [15]:
!ls my_bot/slotfill

dstc_slot_vals.json


In [16]:
!head -n 10 my_bot/slotfill/dstc_slot_vals.json

{
    "food": {
        "caribbean": [
            "carraibean",
            "carribean",
            "caribbean"
        ],
        "kosher": [
            "kosher"
        ],


##### Metric scores on valid&test

Let's check performance of our slot filler on DSTC-2 dataset:

In [17]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

slotfill_config = read_json(configs.ner.slotfill_simple_dstc2_raw)

We take [original DSTC2 slot-filling config](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/ner/slotfill_dstc2_raw.json) and change variables determining data paths:

In [18]:
slotfill_config['metadata']['variables']['DATA_PATH'] = 'my_data'
slotfill_config['metadata']['variables']['SLOT_VALS_PATH'] = 'my_bot/slotfill/dstc_slot_vals.json'

In [19]:
from deeppavlov import evaluate_model

slotfill = evaluate_model(slotfill_config);

2019-09-05 20:02:29.358 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-trn.json]
2019-09-05 20:02:29.363 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-val.json]
2019-09-05 20:02:29.449 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-tst.json]
2019-09-05 20:02:29.537 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 479 samples in train split.
2019-09-05 20:02:29.537 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 6231 samples in valid split.
2019-09-05 20:02:29.538 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 6345 sampl

{"valid": {"eval_examples_count": 1444, "metrics": {"slots_accuracy": 0.9307}, "time_spent": "0:00:27"}}
{"test": {"eval_examples_count": 1386, "metrics": {"slots_accuracy": 0.9481}, "time_spent": "0:00:25"}}


We've got slot accuracy of **93% on valid** set and **94% on test** set.

##### Interacting with slot filler

In [20]:
from deeppavlov import build_model

slotfill = build_model(slotfill_config)

In [21]:
slotfill(['i want cheap chinee food'])

[{'food': 'chinese', 'pricerange': 'cheap'}]

##### Dumping slot filler's config

Saving slotfill config file to disk (we will require it's path later):

In [22]:
import json

json.dump(slotfill_config, open('my_bot/slotfill_config.json', 'wt'))

In [23]:
!ls my_bot

db.sqlite  slotfill  slotfill_config.json


## 3. Train bot

Let's assemble all modules together and train the final module: dialogue policy network.

&nbsp;
![gobot_policy.png](img/gobot_policy.png)
&nbsp;

Policy network decides which action the system should take on each turn of a dialogue: should it say goodbye, request user's location or make api call to a database.

The policy network is a recurrent neural network (recurrent over utterances represented as bags of words) and a dense layer with softmax function on top. The network classifies user utterance into one of predefined system actions.

&nbsp;
![gobot_templates.png](img/gobot_templates.png)
&nbsp;

All actions available for the system should be listed in a `simple-dstc2-templates.txt` file. Each action should be associated with a string of the corresponding system response.

Templates should be in the format `<act>TAB<template>`, where `<act>` is a dialogue action and `<template>` is the corresponding response. Response text might contain slot type names, where every `#slot_type` will be filled with the slot value from a dialogue state.

In [24]:
!head -n 10 my_data/simple-dstc2-templates.txt

api_call	api_call area="#area" food="#food" pricerange="#pricerange"
bye	You are welcome!
canthear	Sorry, I can't hear you.
canthelp_area	I'm sorry but there is no #area american restaurant in the #area of town.
canthelp_area_food	Sorry there is no #food restaurant in the #area of town.
canthelp_area_food_pricerange	Sorry there is no #pricerange restaurant in the #area of town serving #food food.
canthelp_area_pricerange	Sorry there is no #pricerange restaurant in the #area of town serving #area american food.
canthelp_food	I am sorry but there is no #food restaurant that matches your request.
canthelp_food_pricerange	Sorry there is no #food restaurant in the #pricerange price range.
confirm-domain	You are looking for a restaurant is that right?


So, actions are actually classes we classify over. And `simple-dstc2-templates.txt` contains the set of classes.

To train the dialogue policy network for classification task you need action label for each system utterance in training dialogues. The DSTC-2 contains `"act"` dictionary key that contains action associated with current response.

The cell below provides an example of training data for the policy network.

In [25]:
!head -n 24 my_data/simple-dstc2-trn.json

[
  [
    {
      "speaker": 2,
      "text": "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?",
      "slots": [],
      "act": "welcomemsg"
    },
    {
      "speaker": 1,
      "text": "cheap restaurant",
      "slots": [
        [
          "pricerange",
          "cheap"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What kind of food would you like?",
      "slots": [],
      "act": "request_food"
    },


Let's **construct the final pipeline** of a dialogue system.

We take [default DSTC2 bot config](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot/gobot_dstc2.json) ([more configs](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot) are available) and change sections responsible for 
- templates,
- database,
- slot filler,
- embeddings,
- data and model load/save paths.

In [26]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_simple_dstc2)

**Configure** bot to use **templates**:

In [27]:
gobot_config['chainer']['pipe'][-1]['template_type'] = 'DefaultTemplate'
gobot_config['chainer']['pipe'][-1]['template_path'] = 'my_data/simple-dstc2-templates.txt'

**Configure** bot to use our built **database**:

In [28]:
gobot_config['chainer']['pipe'][-1]['database'] = {
    'class_name': 'sqlite_database',
    'primary_keys': ["name"],
    'save_path': 'my_bot/db.sqlite'
}

**Configure** bot to use levenshtein distance based **slot filler**:

In [29]:
gobot_config['chainer']['pipe'][-1]['slot_filler']['config_path'] = 'my_bot/slotfill_config.json'

gobot_config['chainer']['pipe'][-1]['tracker']['slot_names'] = ['pricerange', 'this', 'area', 'food']

You can use a simple **bag-of-words as embedder** (by default):

In [30]:
gobot_config['chainer']['pipe'][-1]['embedder'] = None

Specify train/valid/test **data path** and **path to save** the final bot model:

In [31]:
gobot_config['metadata']['variables']['DATA_PATH'] = 'my_data'

gobot_config['metadata']['variables']['MODEL_PATH'] = 'my_bot'

The whole dialogue system pipeline looks like this:
    
![gobot_pipeline.png](img/gobot_pipeline.png)

##### Training policy network

In [33]:
from deeppavlov import train_model

gobot_config['train']['batch_size'] = 8 # set batch size
gobot_config['train']['max_batches'] = 120 # maximum number of training batches
gobot_config['train']['val_every_n_batches'] = 40 # evaluate on full 'valid' split each 40 batches
gobot_config['train']['log_every_n_batches'] = 40 # evaluate on 20 batches of 'train' split every 4g0 batches
gobot_config['train']['log_on_k_batches'] = 20

train_model(gobot_config);

2019-09-05 20:04:33.346 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-trn.json]
I0905 20:04:33.346428 139952806844032 dstc2_reader.py:290] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-trn.json]
2019-09-05 20:04:33.351 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-val.json]
I0905 20:04:33.351133 139952806844032 dstc2_reader.py:290] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-val.json]
2019-09-05 20:04:33.387 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-tst.json]
I0905 20:04:33.387331 139952806844032 dstc2_reader.py:290] [loading dialogs from /home/vimary/code

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0266}, "time_spent": "0:01:21", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 50, "metrics": {"per_item_dialog_accuracy": 0.5595}, "time_spent": "0:02:00", "epochs_done": 5, "batches_seen": 40, "train_examples_seen": 290, "learning_rate": 0.003, "momentum": 0.95, "loss": 1.5118535563349724}}


2019-09-05 20:07:53.480 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.442
I0905 20:07:53.480249 139952806844032 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.442
2019-09-05 20:07:53.481 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0905 20:07:53.481023 139952806844032 nn_trainer.py:166] Saving model
2019-09-05 20:07:53.481 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0905 20:07:53.481976 139952806844032 tf_model.py:76] [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
2019-09-05 20:07:53.556 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 757: [saving parameters to /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0905 20:07:53.556905 139952806844032 network.py:757] [saving parameters to /home/vimary/code-projects/Pilot/example

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.442}, "time_spent": "0:03:19", "epochs_done": 5, "batches_seen": 40, "train_examples_seen": 290, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 50, "metrics": {"per_item_dialog_accuracy": 0.7516}, "time_spent": "0:03:58", "epochs_done": 11, "batches_seen": 80, "train_examples_seen": 574, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.6322784662246704}}


2019-09-05 20:09:50.864 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.4824
I0905 20:09:50.864818 139952806844032 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.4824
2019-09-05 20:09:50.865 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0905 20:09:50.865603 139952806844032 nn_trainer.py:166] Saving model
2019-09-05 20:09:50.866 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0905 20:09:50.866755 139952806844032 tf_model.py:76] [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
2019-09-05 20:09:50.945 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 757: [saving parameters to /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0905 20:09:50.945839 139952806844032 network.py:757] [saving parameters to /home/vimary/code-projects/Pilot/examp

...

2019-09-05 20:15:50.402 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 296: Stopped training
I0905 20:15:50.402140 139952806844032 nn_trainer.py:296] Stopped training
2019-09-05 20:15:50.416 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
I0905 20:15:50.416077 139952806844032 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
2019-09-05 20:15:50.418 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
W0905 20:15:50.418551 139952806844032 serializable.py:47] No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-05 20:15:50.420 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /home/vimary/code-projects/Pilot/examples

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.5134}, "time_spent": "0:01:22"}}


2019-09-05 20:18:30.800 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
I0905 20:18:30.800374 139952806844032 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
2019-09-05 20:18:30.801 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
W0905 20:18:30.801774 139952806844032 serializable.py:47] No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-05 20:18:30.802 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
I0905 20:18:30.802565 139952806844032 sqlite_database.py:66] Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
2019-09-05 20:18:30.888 IN

{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.5006}, "time_spent": "0:01:19"}}


2019-09-05 20:18:31.509 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 248: [initializing `GoalOrientedBot` from saved]
I0905 20:18:31.509837 139952806844032 network.py:248] [initializing `GoalOrientedBot` from saved]
2019-09-05 20:18:31.510 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 763: [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0905 20:18:31.510730 139952806844032 network.py:763] [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
2019-09-05 20:18:31.513 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0905 20:18:31.513499 139952806844032 tf_model.py:52] [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]


Training on 50 dialogues takes from 5 to 20 minutes depending on gpu/cpu. Training on full data takes 10-30 mins.

See [config doc page](http://docs.deeppavlov.ai/en/master/intro/configuration.html) for advanced configuration of the training process.

##### Metric scores on valid&test

Calculating **accuracy** of trained bot: whether predicted system responses match true responses (full string match).

In [34]:
from deeppavlov import evaluate_model

evaluate_model(gobot_config);

2019-09-05 20:18:31.552 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-trn.json]
I0905 20:18:31.552329 139952806844032 dstc2_reader.py:290] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-trn.json]
2019-09-05 20:18:31.557 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-val.json]
I0905 20:18:31.557561 139952806844032 dstc2_reader.py:290] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-val.json]
2019-09-05 20:18:31.588 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/simple-dstc2-tst.json]
I0905 20:18:31.588336 139952806844032 dstc2_reader.py:290] [loading dialogs from /home/vimary/code

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.5134}, "time_spent": "0:01:19"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.5006}, "time_spent": "0:01:17"}}


With settings of `max_batches=120`, valid accuracy `= 0.5` and test accuracy is `~ 0.5`.

##### Chatting with bot

In [35]:
from deeppavlov import build_model

bot = build_model(gobot_config)

2019-09-05 20:21:07.16 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
I0905 20:21:07.016307 139952806844032 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
2019-09-05 20:21:07.17 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
W0905 20:21:07.017714 139952806844032 serializable.py:47] No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-05 20:21:07.19 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
I0905 20:21:07.019069 139952806844032 sqlite_database.py:66] Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
2019-09-05 20:21:07.104 INFO 

In [36]:
bot(['hi, i want to eat, can you suggest a place to go?'])

['Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?']

In [37]:
bot(['i want cheap food'])

['What kind of food would you like?']

In [38]:
bot(['chinese food'])

2019-09-05 20:21:07.720 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 485: Made api_call with {'pricerange': 'cheap', 'food': 'chinese'}, got 4 results.
I0905 20:21:07.720231 139952806844032 network.py:485] Made api_call with {'pricerange': 'cheap', 'food': 'chinese'}, got 4 results.


['The lucky star serves chinese food.']

In [39]:
bot(['thanks, give me their address'])

['Sure, the lucky star is on cambridge leisure park clifton way cherry hinton.']

In [40]:
bot(['i want their phone number too'])

['The phone number of the lucky star is 01223 244277.']

In [41]:
bot(['bye'])

['Can I help you with anything else?']

In [42]:
bot.reset()

In [43]:
bot(['hi, is there any cheap restaurant?'])

['What kind of food would you like?']